# Практика №4

Теперь мы построим и обучим простую end-to-end модель. Будем работать с пропатченной версией уже готового [пайплайна](https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch). Также нам пригодится [ESPnet](https://github.com/espnet/espnet) для использования модели [Transformer](http://jalammar.github.io/illustrated-transformer/) в качестве энкодера.

### Bootstrap

In [1]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!gdown --id '1skrVbNyrhBLeceGS9CV9uIw_gvo1JiA6'

!unzip -q lab4.zip
!rm -rf lab4.zip sample_data
%cd lab4

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1skrVbNyrhBLeceGS9CV9uIw_gvo1JiA6
To: /content/lab4.zip
100% 2.77M/2.77M [00:00<00:00, 178MB/s]
replace lab4/train_clean_100_text_clean.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
/content/lab4


In [3]:
import os
import torch
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
import math

from utils import TextTransform
from utils import cer
from utils import wer

from espnet.nets.pytorch_backend.transformer.embedding import PositionalEncoding
# from espnet.nets.pytorch_backend.transformer.encoder_layer import EncoderLayer
from espnet.nets.pytorch_backend.conformer.encoder_layer import EncoderLayer
from espnet.nets.pytorch_backend.transformer.repeat import repeat
from espnet.nets.pytorch_backend.transformer.attention import MultiHeadedAttention
from espnet.nets.pytorch_backend.transformer.positionwise_feed_forward import PositionwiseFeedForward
from espnet.nets.pytorch_backend.transformer.layer_norm import LayerNorm
from espnet.nets.pytorch_backend.nets_utils import make_pad_mask

In [4]:
!nvidia-smi

Tue Jun  7 09:50:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
train_audio_transforms = torch.nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=400, hop_length=160, n_mels=80),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram(sample_rate=16000,
                                                              n_fft=400,
                                                              hop_length=160,
                                                              n_mels=80)

text_transform = TextTransformBPE(
    path_to_text="/content/lab4/train_clean_100_text_clean.txt"
)

#-----------------------------TODO №2-----------------------------------
# Заменить графемный токенайзер на сабвордовый TextTransformBPE
#-----------------------------------------------------------------------


def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance))
        labels.append(label)
        input_lengths.append(spec.shape[0])
        label_lengths.append(len(label))

    spectrograms = torch.nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths


def GreedyDecoder(output, labels, label_lengths, blank_label=2000, collapse_repeated=True):

    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [8]:
class TransformerModel(torch.nn.Module):

    def __init__(
        self,
        input_size=80,
        output_size=2001,
        conv2d_filters=32,
        attention_dim=240,
        attention_heads=8,
        feedforward_dim=512,
        num_layers=8,
        dropout=0.1,
    ):
        super(TransformerModel, self).__init__()
        
        self.conv_in = torch.nn.Sequential(
            torch.nn.Conv2d(1, conv2d_filters, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            # torch.nn.BatchNorm2d(conv2d_filters),
            torch.nn.ReLU(),
            torch.nn.Conv2d(conv2d_filters, conv2d_filters, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            # torch.nn.BatchNorm2d(conv2d_filters),
            torch.nn.ReLU(),
            torch.nn.Conv2d(conv2d_filters, conv2d_filters, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            # torch.nn.BatchNorm2d(conv2d_filters),
            torch.nn.ReLU(),
        )
        self.conv_out = torch.nn.Sequential(
            torch.nn.Linear(conv2d_filters * (input_size // 8), attention_dim),
            PositionalEncoding(attention_dim, 0.1),
        )
        positionwise_layer = PositionwiseFeedForward
        positionwise_layer_args = (attention_dim, feedforward_dim, dropout)
        self.encoder_layer = repeat(
            num_layers,
            lambda lnum: EncoderLayer(
                attention_dim,
                MultiHeadedAttention(
                    attention_heads, attention_dim, dropout
                ),
                positionwise_layer(*positionwise_layer_args),
                positionwise_layer(*positionwise_layer_args),  # feed_forward_macaron
                None,
                dropout,
                normalize_before=True,
                concat_after=False,
            ),
        )
        self.after_norm = LayerNorm(attention_dim)
        self.final_layer = torch.nn.Linear(attention_dim, output_size)

    def forward(self, x, ilens):
        x = x.unsqueeze(1)  # (b, c, t, f)
        x = self.conv_in(x)
        b, c, t, f = x.size()
        x = self.conv_out(x.transpose(1, 2).contiguous().view(b, t, c * f))
        masks = (~make_pad_mask(ilens)[:, None, :])[:, :, ::8].to(x.device)
        x, _, _ = self.encoder_layer(x, masks)
        x = self.after_norm(x)
        x = self.final_layer(x)
        return x

In [9]:
x = torch.rand(2, 800, 80)
model = TransformerModel()
output = model(x, [800, 90])
print(output.shape)

torch.Size([2, 100, 2001])


In [10]:
def train(model, device, train_loader, criterion, optimizer, scheduler, epoch):
    model.train()
    data_len = len(train_loader.dataset)

    for batch_idx, _data in enumerate(train_loader):
        spectrograms, labels, input_lengths, label_lengths = _data 
        spectrograms, labels = spectrograms[:, :, :,:max(input_lengths)].to(device), labels.to(device) #(batch, 1, feat_dim, time)
        spectrograms = spectrograms.squeeze(1).transpose(1,2) # (batch, time, feat_dim,)
        optimizer.zero_grad()
        
        output = model(spectrograms, input_lengths)  # (batch, time, n_classes)
        output = F.log_softmax(output, dim=2)
        output = output.transpose(0, 1) # (time, batch, n_class)
        input_lengths = [x // 8 for x in input_lengths]

        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        scheduler.step()
        if batch_idx % 500 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tLR: {:.6f}'.format(
                epoch,
                batch_idx * len(spectrograms),
                data_len,
                100. * batch_idx / len(train_loader),
                loss.item(),
                scheduler.get_last_lr()[0]))


def test(model, device, test_loader, criterion, epoch, decode=True):
    print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)
            spectrograms = spectrograms.squeeze(1).transpose(1,2) # (batch time, feat_dim,)
            
            output = model(spectrograms, input_lengths)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)
            input_lengths = [x // 8 for x in input_lengths]

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            if decode:
              decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels.int(), label_lengths)
              for j in range(len(decoded_preds)):
                  test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                  test_wer.append(wer(decoded_targets[j], decoded_preds[j]))
    if decode:
        avg_cer = sum(test_cer)/len(test_cer)
        avg_wer = sum(test_wer)/len(test_wer)

        print(f"Test set: Average loss: {test_loss:.4f}, Average CER: {avg_cer:4f} Average WER: {avg_wer:.4f}\n")
    else:
        print(f"Average loss: {test_loss:.4f}\n")

In [11]:
def main(learning_rate=1e-5, batch_size=20, test_batch_size=7, epochs=10,
        train_url="train-clean-100", test_url="test-clean"):
    
    hparams = {
        "input_size": 80,
        "output_size": 2001,
        "conv2d_filters": 32,
        "attention_dim": 512,  # 320,
        "attention_heads": 8,
        "feedforward_dim": 2048,  # 1024,
        "num_layers": 12,  # 10
        "dropout": 0.1,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs
    }

    use_cuda = torch.cuda.is_available()
    torch.manual_seed(7)
    device = torch.device("cuda" if use_cuda else "cpu")

    if not os.path.isdir("./data"):
        os.makedirs("./data")

    train_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=train_url, download=True)
    test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=test_url, download=True)

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'train'),
                                **kwargs)
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=test_batch_size,
                                shuffle=False,
                                collate_fn=lambda x: data_processing(x, 'valid'),
                                **kwargs)

    model = TransformerModel(
        hparams['input_size'],
        hparams['output_size'],
        hparams['conv2d_filters'],
        hparams['attention_dim'],
        hparams['attention_heads'],
        hparams['feedforward_dim'],
        hparams['num_layers'],
        hparams['dropout']).to(device)

    print(model)
    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = torch.nn.CTCLoss(blank=2000, zero_infinity=False).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=hparams['epochs'],
                                            anneal_strategy='linear')
    
    for epoch in range(1, epochs + 1):
        !date
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch)
        test(model, device, test_loader, criterion, epoch, decode=not(epoch % 5))

In [12]:
learning_rate = 1e-3
batch_size = 16
test_batch_size = 8
epochs = 10
libri_train_set = "train-clean-100"
libri_test_set = "test-clean"

main(learning_rate, batch_size, test_batch_size, epochs, libri_train_set, libri_test_set)

  0%|          | 0.00/5.95G [00:00<?, ?B/s]

  0%|          | 0.00/331M [00:00<?, ?B/s]

TransformerModel(
  (conv_in): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
  )
  (conv_out): Sequential(
    (0): Linear(in_features=320, out_features=512, bias=True)
    (1): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (encoder_layer): MultiSequential(
    (0): EncoderLayer(
      (self_attn): MultiHeadedAttention(
        (linear_q): Linear(in_features=512, out_features=512, bias=True)
        (linear_k): Linear(in_features=512, out_features=512, bias=True)
        (linear_v): Linear(in_features=512, out_features=512, bias=True)
        (linear_out): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): PositionwiseFeedForward(
        (

### <b>Задание №1</b> (5 баллов):
На данный момент практически все E2E SOTA решения используют [сабворды](https://dyakonov.org/2019/11/29/%D1%82%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F-%D0%BD%D0%B0-%D0%BF%D0%BE%D0%B4%D1%81%D0%BB%D0%BE%D0%B2%D0%B0-subword-tokenization/) (subwords/wordpieces) в качестве таргетов нейронки для распознавания. Нам бы тоже не мешало перейти от графем к сабвордам. Теперь вместо букв (графем) будем распознавать кусочки слов. В качестве такого токенайзера предлагается использовать [Sentencepiece](https://github.com/google/sentencepiece). Пример обучения BPE токенайзера можно найти в [link](https://github.com/google/sentencepiece/tree/master/python). Главное правильно обернуть его в наш класс TextTransformBPE. Текстовый файл (train_clean_100_text_clean.txt) для обучения токенайзера уже подготовлен и лежит в корневой папке проекта. 

In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import sentencepiece as spm


class TextTransformBPE:
    def __init__(self, path_to_text):
        """ Обучение BPE модели на 2000 юнитов."""
        spm.SentencePieceTrainer.train(
            input=path_to_text, 
            model_prefix='m', 
            vocab_size=2000,
            model_type="bpe"
        )
        self.sp = spm.SentencePieceProcessor(
            model_file='m.model'
        )

    def text_to_int(self, text):
        """ Преобразование входного текста в последовательность сабвордов в формате их индекса в BPE модели """
        # int_sequence = self.sp.encode_as_ids(text)
        int_sequence = self.sp.encode(text)
        return int_sequence

    def int_to_text(self, labels):
        """ Преобразование последовательности индексов сабвордов в текст """
        # string = self.sp.decode_ids(labels)
        string = self.sp.decode(labels)
        return string

### **Ответ**:
Код для обучения BPE токенайзера находится в ячейке выше. 

Создание объекта (экзмепляра класса TextTransformBPE) происходит в ячейке **7**. Также в коде были изменены значения параметров **blank** с 28 на 2000 и **output_size** с 29 на 2001.

Результаты (значение WER, CTC и CER) при использовании графем и сабвордов приведены в таблице ниже. Все метрики взяты на конец 10 эпохи на тестовом множестве. Кроме таргетов модели ничем не отличаются.

||WER|CER|CTC|
|-|-|-|-|
|Графемы|0.54|0.18|0.67|
|Сабворды|0.51|0.31|1.95|

WER улучшилось всего на 3%, хотя не покидает чувство, что что-то сделал не так. Т.к. коллеги в чате писали, что у них на 6% улучшилось качество. При этом метрики CER и CTC у меня стали хуже.

### <b>Задание №2</b> (5 баллов):
Импровизация по улучшению качества распознавания.

### **Ответ**:

Здесь приведены изменения в архитектуре модели и результаты, которые удалось получить с этими изменениями. В качестве результатов будет использоваться только значение **WER** на конец 10-й эпохи (один раз запустил на ночь на 20 эпох, но colab благоразумно решил прервать сессию на 11 эпохе)

> **1)** Первоначально добавил в сверточную часть еще одну свертку со страйдом 2, аналогичную предыдущей свертке (*ячейка 8*). Таким образом, размер выход сверточной части уменьшился в 8 раз. И здесь же еще изменил размеры батчей: **bacth_size=24**, **test_batch_size=8**. Полученный результат:
$${WER=0.46}$$
Качество улушилось на **5%**, что хорошо. Поэтому данное изменение в нашей модели сохраним.


> **2)** В дополнении к первому пунтку были изменены параметры трансформера: **attention_dim=512**, **feedforward_dim=2048**, **num_layers=12** и размер батча на обучении взял побольше: **bacth_size=32**. Полученный результат:
$${WER=0.4}$$
Качество улучшилось еще на **6%**.


> **3)** Добавил Conformer вместо Transformer в качестве EncoderLayer (*ячейка 8*). Параметры **feed_forward_macaron** и **conv_module** выставлял равными **None** (как в исходниках значения по умолчанию: https://espnet.github.io/espnet/_modules/espnet/nets/pytorch_backend/conformer/encoder.html). Полученный результат:
$${WER=0.39}$$
Еще на **1%** удалось улучшить.

> **4)** А здесь приведены изменения, которые только ухудшили результат, по сравнению с исходным, полученным в прошлом пункте.
> * Увеличение максимального *learning_rate* до *1e-2*
> * Добавление батч нормализации в сверточных слоях + увеличение *num_layers* до 12.
> * Добавление батч нормализации и увеличение максимального *learning_rate* до *3e-5* (была идея в том, что батч нормализация вроде как должна ускорять сходимость, поэтому можно learning_rate сделать побольше)
> * Conformer в качестве EncoderLayer и **feed_forward_macaron = positionwise_layer(*positionwise_layer_args)**, как все в тех же исходниках. Но **conv_module = None**.

> **5)** Вероятно, увеличение количества эпох приведет к улучшению качества. Но при прочих равных на 10 эпохах максимум удалось достичь значения **WER=39%**, что на **15%** лучше, чем самый первый результат.